In [162]:
from pyspark import SparkContext
from pyspark import SparkConf
from operator import add
from pyspark import SparkContext,SQLContext, Row
from pyspark.sql import SQLContext
from pyspark.mllib.recommendation import ALS, Rating

In [184]:
conf = SparkConf().setAppName("book_recommendation-server-2")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=book_recommendation-server, master=local[*]) created by __init__ at <ipython-input-8-869e853a2175>:2 

In [185]:
books_raw_RDD = sc.textFile('main/BX-CSV-Dump/BX-Books.csv')

In [186]:
books_raw_RDD.take(1)

['"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"']

In [187]:
books_raw_data_header

'"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"'

In [190]:
books_RDD = books_raw_RDD.filter(lambda line: line != '"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"')\
                            .map(lambda line: line.split(';'))\
                            .map(lambda tokens: (abs(hash(tokens[0][1:-1])) % (10 ** 8), tokens[1][1:-1], tokens[2][1:-1], tokens[3][1:-1], tokens[4][1:-1], tokens[5][1:-1]))\
                            .cache()

In [191]:
books_RDD.take(1)

[(85973176,
  'Classical Mythology',
  'Mark P. O. Morford',
  '2002',
  'Oxford University Press',
  'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg')]

In [192]:
books_titles_RDD = books_RDD.map(lambda x : (int(x[0]), x[1], x[2], x[3], x[4], x[5])).cache()

In [193]:
books_titles_RDD.take(1)

[(85973176,
  'Classical Mythology',
  'Mark P. O. Morford',
  '2002',
  'Oxford University Press',
  'http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg')]

In [194]:
ratings_raw_RDD = sc.textFile('main/BX-CSV-Dump/BX-Book-Ratings.csv')

In [195]:
ratings_raw_data_header = ratings_raw_RDD.take(1)[0]

In [196]:
ratings_raw_data_header

'"User-ID";"ISBN";"Book-Rating"'

In [197]:
ratings_RDD = ratings_raw_RDD.filter(lambda line : line != ratings_raw_data_header) \
                                .map(lambda line : line.split(';'))\
                                .map(lambda tokens: (int(tokens[0][1:-1]), hash(tokens[1][1:-1]) % 10 ** 8, int(tokens[2][1:-1])))

In [198]:
ratings_RDD.take(10)

[(276725, 68701106, 0),
 (276726, 46426205, 5),
 (276727, 90758053, 0),
 (276729, 44987036, 3),
 (276729, 78989836, 6),
 (276733, 30122732, 0),
 (276736, 64479825, 8),
 (276737, 43822043, 6),
 (276744, 83371955, 7),
 (276745, 78660650, 10)]

In [199]:
book_ID_with_ratings_RDD = ratings_RDD.map(lambda x: (x[1], x[2])).groupByKey()

In [200]:
for x in book_ID_with_ratings_RDD.take(2)[0][1]:
    print(x)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 111.0 failed 1 times, most recent failure: Lost task 0.0 in stage 111.0 (TID 103, localhost, executor driver): java.io.FileNotFoundException: C:\Users\username\AppData\Local\Temp\blockmgr-0396aab9-e6cb-4384-80f6-b3aed9351501\0d\temp_shuffle_6b4fd8c7-d553-441c-8055-758871f3d880 (系统找不到指定的路径。)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWrit

In [201]:
def get_counts_and_average(ID_and_ratings_tuple):
    n = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (n, float(sum(x for x in ID_and_ratings_tuple[1]))/ n)

In [202]:
book_ID_with_avg_ratings_RDD = book_ID_with_ratings_RDD.map(get_counts_and_average)

In [203]:
book_ID_with_avg_ratings_RDD.take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 113.0 failed 1 times, most recent failure: Lost task 0.0 in stage 113.0 (TID 105, localhost, executor driver): java.io.FileNotFoundException: C:\Users\username\AppData\Local\Temp\blockmgr-0396aab9-e6cb-4384-80f6-b3aed9351501\11\temp_shuffle_83d4a834-7c19-41ae-9ada-be99b116effa (系统找不到指定的路径。)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWrit

In [204]:
book_ratings_count_RDD = book_ID_with_avg_ratings_RDD.map(lambda x : (x[0], x[1][0]))

In [205]:
book_ratings_count_RDD.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 115.0 failed 1 times, most recent failure: Lost task 0.0 in stage 115.0 (TID 107, localhost, executor driver): java.io.FileNotFoundException: C:\Users\username\AppData\Local\Temp\blockmgr-0396aab9-e6cb-4384-80f6-b3aed9351501\36\temp_shuffle_a2bf9e53-7e28-4809-8fc6-f776986d2a57 (系统找不到指定的路径。)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWrit

In [ ]:
ratings_RDD.take(2)

In [ ]:
rank = 16
seed = 5
iterations = 10
regularization_parameter = 0.1

In [ ]:
model = ALS.train(ratings_rdd, rank=rank, seed=seed, iterations=iterations, lambda_=regularization_parameter )

In [ ]:
model.take(1)

In [ ]:
user_unrated_books_RDD = ratings_RDD.filter(lambda x : x[0] != 276725).map(lambda x : (276725, x[1])).distinct()

In [ ]:
user_unrated_books_RDD.take(9)

In [ ]:
predicted_RDD = model.predictAll(user_unrated_books_RDD)

In [ ]:
len(predicted_RDD.take(2))

In [ ]:
predicted_rating_RDD = predicted_RDD.map(lambda x: (x.product, x.rating))

In [ ]:
predicted_rating_RDD.takeOrdered(5, key=lambda x: -x[1])

In [ ]:
sqlContext=SQLContext(sc)

In [ ]:
mydf001=sqlContext.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/books_recommend")\
  .option("dbtable","main_book").option("user","root").option("password","password").load()

In [ ]:
type(mydf001)

In [ ]:
sqlContext

In [ ]:
rd = mydf001.rdd

In [ ]:
(rd.take(10)[0])

In [ ]:
rd = mydf001.rdd.map(lambda x: (x.id, x.title, x.published_year, x.isbn))

In [ ]:
rd = rd.cache()

In [ ]:
rd.count()

In [ ]:
rd = rd.map(lambda x : (x[2], (x[0], x[1], x[2], x[3])))

In [ ]:
rd.take(300)

In [ ]:
rd.count()

In [ ]:
rds = rd.sortByKey(False)

In [ ]:
rds.take(30)

In [ ]:
rds.take(3)

In [ ]:
ratings_df=sqlContext.read.format("jdbc").option("url","jdbc:mysql://localhost:3306/books_recommend")\
  .option("dbtable","main_rating").option("user","root").option("password","password").load()

In [ ]:
ratings_sql_rdd = ratings_df.rdd

In [ ]:
ratings_rdd = ratings_sql_rdd.map(lambda x : (x.user_id, hash(x.isbn) % 10 ** 9, x.rating))

In [ ]:
ratings_rdd = ratings_rdd.map(lambda x : (x[0], x[1], x[2]))

In [ ]:
ratings_rdd.take(100)

In [ ]:
sc.setCheckpointDir('D:/checkpoint/')
ALS.checkpointInterval = 2

In [ ]:
model = ALS.train(ratings=ratings_rdd, rank=20, iterations=5, lambda_=0.02)

In [ ]:
model.predit(276798)